In [5]:
import cv2
import numpy as np
from keras.models import load_model
from statistics import mode
from datasets import get_labels
from inference import detect_faces
from inference import draw_text
from inference import draw_bounding_box
from inference import apply_offsets
from inference import load_detection_model
from preprocessor import preprocess_input
import matplotlib.pyplot as plt
imgf=cv2.imread('fear.jpg',1)
imgs=cv2.imread('sad.jpg',1 )
imgh=cv2.imread('happy.jpg',1 )
imgn=cv2.imread('neutral.jpg',1 )
imgd=cv2.imread('disgust.jpg',1 )
imgsu=cv2.imread('surprise.jpg',1 )
imga=cv2.imread('angry.jpg',1 )
USE_WEBCAM = True # If false, loads video file source
# parameters for loading data and images
emotion_model_path = ('C:\\Users\\user\\Desktop\\ismriti\\Emotion-master\\models\\best_weights_model.hdf5')
emotion_labels = get_labels('fer2013')

# hyper-parameters for bounding boxes shape
frame_window = 100
emotion_offsets = (20, 40)

# loading models
face_cascade = cv2.CascadeClassifier('C:\\Users\\user\\Desktop\\ismriti\\Emotion-master\\models\\haarcascade_frontalface_default.xml')
emotion_classifier = load_model(emotion_model_path)

# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]

# starting lists for calculating modes
emotion_window = []

# starting video streaming

cv2.namedWindow('window_frame')
video_capture = cv2.VideoCapture(0)
def gr(l,b):
    em=['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    index = np.arange((7))
    plt.figure()
    plt.bar(index, list(l))
    plt.xlabel('Emotions', fontsize=10)
    plt.ylabel('probability', fontsize=10)
    plt.xticks(index, em, fontsize=10, rotation=30)
    plt.title('Distribution of emotions')
    plt.show()
# Select video or webcam feed
cap = None
if (USE_WEBCAM == True):
    cap = cv2.VideoCapture(0) # Webcam source
else:
    cap = cv2.VideoCapture('./demo/dinner.mp4') # Video file source

while cap.isOpened(): # True:
    ret, bgr_image = cap.read()

    #bgr_image = video_capture.read()[1]

    gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)

    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5,
minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    for face_coordinates in faces:

        x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
        gray_face = gray_image[y1:y2, x1:x2]
        try:
            gray_face = cv2.resize(gray_face, (emotion_target_size))
        except:
            continue

        gray_face = preprocess_input(gray_face, True)
        gray_face = np.expand_dims(gray_face, 0)
        gray_face = np.expand_dims(gray_face, -1)
        emotion_prediction = emotion_classifier.predict(gray_face)
        emotion_probability = np.max(emotion_prediction)
        emotion_label_arg = np.argmax(emotion_prediction)
        emotion_text = emotion_labels[emotion_label_arg]
        emotion_window.append(emotion_text)

        a=emotion_prediction.flatten()
        gr(a,emotion_probability)
        if len(emotion_window) > frame_window:
            emotion_window.pop(0)
        try:
            emotion_mode = mode(emotion_window)
        except:
            continue

        if emotion_text == 'angry':
            color = emotion_probability * np.asarray((255, 0, 0))
            cv2.imshow('image',imga)
        elif emotion_text == 'sad':
            color = emotion_probability * np.asarray((0, 0, 255))
            cv2.imshow('image',imgs)
        elif emotion_text == 'happy':
            color = emotion_probability * np.asarray((255, 255, 0))
            cv2.imshow('image',imgh)
        elif emotion_text == 'surprise':
            color = emotion_probability * np.asarray((0, 255, 255))
            cv2.imshow('image',imgsu)
        elif emotion_text=='fear':
            color = emotion_probability * np.asarray((200, 150, 100))
            cv2.imshow('image',imgf)
        elif emotion_text=='neutral':
            color = emotion_probability * np.asarray((182, 13, 85))
            cv2.imshow('image',imgn)
        else:
            color = emotion_probability * np.asarray((200, 200, 0))
            cv2.imshow('image',imgd)

        color = color.astype(int)
        color = color.tolist()
        
       
        draw_bounding_box(face_coordinates, rgb_image, color)
        draw_text(face_coordinates, rgb_image, emotion_mode,
                  color, 0, -45, 1, 1)

    bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
    cv2.imshow('window_frame', bgr_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-5b30149766b2>", line 31, in <module>
    emotion_classifier = load_model(emotion_model_path)
  File "C:\Users\user\Anaconda3\lib\site-packages\keras\engine\saving.py", line 417, in load_model
    f = h5dict(filepath, 'r')
  File "C:\Users\user\Anaconda3\lib\site-packages\keras\utils\io_utils.py", line 186, in __init__
    self.data = h5py.File(path, mode=mode)
  File "C:\Users\user\Anaconda3\lib\site-packages\h5py\_hl\files.py", line 394, in __init__
    swmr=swmr)
  File "C:\Users\user\Anaconda3\lib\site-packages\h5py\_hl\files.py", line 170, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py\_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py\_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py\h5f.pyx", line 8

OSError: Unable to open file (unable to open file: name = 'C:\Users\user\Desktop\ismriti\Emotion-master\models\best_weights_model.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)